<a href="https://colab.research.google.com/github/dmarcus9/Project-3.SnakesID/blob/master/7_Training_Vgg19.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [41]:
# Use colab to make this faster...running on cloud...tensorflow & keras are pre-loaded
# Colaboratory is a research tool for machine learning education and research. 
# It’s a Jupyter notebook environment that requires no setup to use.

# linked to google drive...each time I will authorize with code
# this is the path we'll use:   /content/drive

# Load the Drive helper and mount
from google.colab import drive

# This will prompt for authorization.
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [0]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import glob
import os
import time

from keras.applications.vgg19 import (
    VGG19, preprocess_input, decode_predictions)

from keras.preprocessing import image
from keras.layers import Dense, Activation, Flatten
from keras.layers import merge, Input
from keras.models import Model
from keras.utils import to_categorical, np_utils
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping

In [0]:
def process_image(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    return x

In [0]:
# this is the path we'll use:   /content/drive/...

train_path = "/content/drive/My Drive/Resources/train"
test_path = "/content/drive/My Drive/Resources/test"
names = ['Copperhead','Coral','CottonMouth','RattlerDB']
names_map = {name: i for i,name in enumerate(names)}
num_classes = len(names)

In [10]:
def get_images(train_or_test_path):
    img_data_list=[]
    labels = []
    for dataset in os.listdir(train_or_test_path):
        img_list = glob.glob(train_path + "/" + dataset + "/*")
# img_list = glob.glob(train_path + "/*")
        print ('Loaded the images of dataset-'+'{}\n'.format(dataset))
        for img in img_list:
            img_path = img
            img = process_image(img_path)
            img_data_list.append(img)
            labels.append(names_map[dataset.lower()])
    img_data = np.array(img_data_list)
    print (img_data.shape)
    img_data=np.rollaxis(img_data,1,0)
    print (img_data.shape)
    img_data=img_data[0]
    print (img_data.shape)
    # convert class labels to one-hot encoding
    Y = np_utils.to_categorical(labels, num_classes)
    return [img_data, Y]

img_data, Y = get_images(train_path)

OSError: ignored

In [0]:
# Define the number of classes
num_of_samples = img_data.shape[0]

#Shuffle the dataset
x,y = shuffle(img_data,Y, random_state=2)
# Split the dataset...20% of the training images are used to validate
X_train, X_valid, y_train, y_valid = train_test_split(x, y, test_size=0.2, random_state=2)

NameError: ignored

In [0]:
#########################################################################################
# Custom_vgg_model_1
#Training the classifier alone
image_input = Input(shape=(224, 224, 3))

model = VGG19(input_tensor=image_input, include_top=True,weights='imagenet')
model.summary()

# grabs last layer
last_layer = model.get_layer('fc2').output

# changes to our categories
out = Dense(num_classes, activation='softmax', name='output')(last_layer)
custom_vgg_model = Model(image_input, out)

for layer in custom_vgg_model.layers[:-1]:
    layer.trainable = False

custom_vgg_model.summary()
    
custom_vgg_model.layers[3].trainable

custom_vgg_model.compile(loss='categorical_crossentropy',optimizer='rmsprop',metrics=['accuracy'])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 224, 224, 3)       0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
__________

In [0]:
# this is where we begin the training...10 epochs?
t=time.time()
#	t = now()

# https://keras.io/callbacks/#earlystopping - if more epochs are not needed.

callbacks = EarlyStopping(monitor='val_loss', min_delta=0, patience=3, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

hist = custom_vgg_model.fit(X_train, y_train, batch_size=32, epochs=3, verbose=1, validation_data=(X_valid, y_valid), callbacks=callbacks)
print('Training time: %s' % (t - time.time()))
(loss, accuracy) = custom_vgg_model.evaluate(X_valid, y_valid, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

# loss = trying to minimize; not a %; sum of errors; 
# acc = accuracy
# val = validation...uses new images...val_loss & acc are where to focus
# overfitting is revealed when acc is not improving, but loss is very close to 0

Train on 323 samples, validate on 81 samples
Epoch 1/3
323/323 [==============================] - 257s 797ms/step - loss: 0.2354 - acc: 0.8947 - val_loss: 0.1733 - val_acc: 0.9506
Epoch 2/3
323/323 [==============================] - 252s 779ms/step - loss: 0.0180 - acc: 1.0000 - val_loss: 0.1577 - val_acc: 0.9506
Epoch 3/3
323/323 [==============================] - 342s 1s/step - loss: 0.0058 - acc: 1.0000 - val_loss: 0.1418 - val_acc: 0.9506
Training time: -851.2848360538483
81/81 [==============================] - 43s 532ms/step
[INFO] loss=0.1418, accuracy: 95.0617%


In [0]:
img_data_test, y_test = get_images(test_path)

Loaded the images of dataset-dogs

Loaded the images of dataset-cats

(404, 1, 224, 224, 3)
(1, 404, 224, 224, 3)
(404, 224, 224, 3)


In [0]:
# this is where the test images are used

(loss, accuracy) = custom_vgg_model.evaluate(img_data_test, y_test, batch_size=10, verbose=1)

print("[INFO] loss={:.4f}, accuracy: {:.4f}%".format(loss,accuracy * 100))

404/404 [==============================] - 221s 547ms/step
[INFO] loss=0.0323, accuracy: 99.0099%


In [0]:
# /content/drive/Project 3/test...
custom_vgg_model.save("dogcat_transfer.h5")

In [0]:
# Load the model
from keras.models import load_model
dog_model = load_model("dogcat_transfer.h5")

In [0]:
dog_model.predict

<bound method Model.predict of <keras.engine.training.Model object at 0x1125c6a20>>